In [1]:
import wandb
import pandas as pd
import numpy as np

In [7]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [21]:
harmonic_mean = lambda xs: len(xs) / sum((x + 1e-6) ** -1 for x in xs)
rm_disasters = ["hurricane-matthew", "santa-rosa-wildfire", "palu-tsunami"]
rm_disasters_to_model_key = dict(zip(rm_disasters,["hurricane", "wildfire", "tsunami"]))

def get_df(run):
    df = run.logged_artifacts()[0].get("per_image_metrics")
    df = pd.DataFrame(df.data, columns=df.columns)
    return df

def get_score_differences_on_subset_data(disaster_type):
    """
    Get differences in mean score, damage and localisation f1 score between a run with and without a certain event of the given disaster type.
    The evaluation is limited to images of type disaster_type.
    """
    base_df = df_dict["base"]
    base_df_subset = base_df[base_df["pre_img"].map(lambda s: s.find(disaster_type) != -1)]
    base_metrics = base_df_subset[["score", "damage_f1", "localization_f1"]].mean()

    disaster_df = df_dict[disaster_type]
    disaster_df_subset = disaster_df[disaster_df["pre_img"].map(lambda s: s.find(disaster_type) != -1)]
    disaster_metrics = disaster_df_subset[["score", "damage_f1", "localization_f1"]].mean()

    return (base_metrics-disaster_metrics)

def compute_f1s(df):   
    f1s = []
    for tp_name in ["lTP", "dTP1", "dTP2", "dTP3", "dTP4"]:
        fn_name = tp_name.replace("TP", "FN")
        fp_name = tp_name.replace("TP", "FP")
        tp = df[tp_name].sum()
        if tp == 0:
            precision = 0
            recall = 0
            f1 = 0
        else:
            precision = tp / (tp + df[fp_name].sum())
            recall = tp / (tp + df[fn_name].sum())
            f1 = 2*precision*recall / (precision + recall)
        f1s.append(f1)
    f1s.append(harmonic_mean(f1s[1:]))
    result = pd.Series(data=f1s, index=["localization_f1", "damage_f1_no_damage", "damage_f1_minor_damage", "damage_f1_major_damage", "damage_f1_destroyed", "damage_f1"])
    result["score"] = result["localization_f1"] * 0.3 + result["damage_f1"] * 0.7
    return result

def get_df_differences_on_subset_data(disaster_type):
    """
    Get differences in mean score, damage and localisation f1 score between a run with and without a certain event of the given disaster type.
    The evaluation is limited to images of type disaster_type.
    """
    base_df = df_dict["base"]
    base_df_subset = base_df[base_df["pre_img"].map(lambda s: s.find(disaster_type) != -1)]
    base_df_f1s = compute_f1s(base_df_subset)

    disaster_df = df_dict[disaster_type]
    disaster_df_subset = disaster_df[disaster_df["pre_img"].map(lambda s: s.find(disaster_type) != -1)]
    disaster_df_f1s = compute_f1s(disaster_df_subset)

    return (base_df_f1s-disaster_df_f1s)

def get_f1s_on_removed_data(disaster_type):
    """
    Get differences in mean score, damage and localisation f1 score between a run with and without a certain event of the given disaster type.
    The evaluation is limited to images of type disaster_type.
    """
    base_df = df_dict["base"]
    base_df_subset = base_df[base_df["pre_img"].map(lambda s: s.find(disaster_type) != -1)]
    base_df_f1s = compute_f1s(base_df_subset)

    disaster_df = df_dict[rm_disasters_to_model_key[disaster_type]]
    disaster_df_subset = disaster_df[disaster_df["pre_img"].map(lambda s: s.find(disaster_type) != -1)]
    disaster_df_f1s = compute_f1s(disaster_df_subset)

    return base_df_f1s, disaster_df_f1s

def get_df_differences_on_removed_data(disaster_type):
    base_df_f1s, disaster_df_f1s = get_f1s_on_removed_data(disaster_type)
    return base_df_f1s - disaster_df_f1s

In [3]:
api = wandb.Api()
base_run = api.run("sgerard/xview2_no1_solution/1vofuk9t")
wildfire_run = api.run("sgerard/xview2_no1_solution/3jj3528p")
hurricane_run = api.run("sgerard/xview2_no1_solution/1l5yb33b")
tsunami_run = api.run("sgerard/xview2_no1_solution/24f7j1fg")

run_dict = {"base":base_run, "hurricane":hurricane_run, "tsunami":tsunami_run, "wildfire":wildfire_run}
df_dict = {name:get_df(run) for name, run in run_dict.items()}

In [20]:
for disaster_type in ["base", "hurricane", "wildfire", "tsunami"]:
    print(disaster_type, "\n\n", compute_f1s(df_dict[disaster_type]), "\n")

base 

 localization_f1          0.86454
damage_f1_no_damage      0.90769
damage_f1_minor_damage   0.49109
damage_f1_major_damage   0.68861
damage_f1_destroyed      0.80971
damage_f1                0.68667
score                    0.74003
dtype: float64 

hurricane 

 localization_f1          0.86410
damage_f1_no_damage      0.90413
damage_f1_minor_damage   0.34079
damage_f1_major_damage   0.71418
damage_f1_destroyed      0.80168
damage_f1                0.59809
score                    0.67789
dtype: float64 

wildfire 

 localization_f1          0.86313
damage_f1_no_damage      0.90992
damage_f1_minor_damage   0.48106
damage_f1_major_damage   0.70962
damage_f1_destroyed      0.80587
damage_f1                0.68636
score                    0.73939
dtype: float64 

tsunami 

 localization_f1          0.86547
damage_f1_no_damage      0.90212
damage_f1_minor_damage   0.45221
damage_f1_major_damage   0.63013
damage_f1_destroyed      0.77551
damage_f1                0.64555
score         

In [15]:
for disaster_type in ["hurricane", "wildfire", "tsunami"]:
    print(disaster_type, "\n", get_df_differences_on_subset_data(disaster_type), "\n")

hurricane 
 localization_f1           0.00154
damage_f1_no_damage      -0.00163
damage_f1_minor_damage    0.15334
damage_f1_major_damage   -0.02804
damage_f1_destroyed       0.01569
damage_f1                 0.06177
score                     0.04370
dtype: float64 

wildfire 
 localization_f1           0.01166
damage_f1_no_damage       0.01953
damage_f1_minor_damage   -0.00133
damage_f1_major_damage   -0.00863
damage_f1_destroyed       0.01731
damage_f1                -0.00459
score                     0.00029
dtype: float64 

tsunami 
 localization_f1          0.00577
damage_f1_no_damage      0.03208
damage_f1_minor_damage   0.00000
damage_f1_major_damage   0.11845
damage_f1_destroyed      0.11128
damage_f1                0.00000
score                    0.00173
dtype: float64 



In [22]:
for disaster_type in ["hurricane-matthew", "santa-rosa-wildfire", "palu-tsunami"]:
    print(disaster_type, "\n", get_df_differences_on_removed_data(disaster_type), "\n")

hurricane-matthew 
 localization_f1          0.04402
damage_f1_no_damage      0.09674
damage_f1_minor_damage   0.68684
damage_f1_major_damage   0.23633
damage_f1_destroyed      0.09378
damage_f1                0.38664
score                    0.28385
dtype: float64 

santa-rosa-wildfire 
 localization_f1           0.01166
damage_f1_no_damage       0.01953
damage_f1_minor_damage   -0.00133
damage_f1_major_damage   -0.00863
damage_f1_destroyed       0.01731
damage_f1                -0.00459
score                     0.00029
dtype: float64 

palu-tsunami 
 localization_f1          0.00577
damage_f1_no_damage      0.03208
damage_f1_minor_damage   0.00000
damage_f1_major_damage   0.11845
damage_f1_destroyed      0.11128
damage_f1                0.00000
score                    0.00173
dtype: float64 



In [43]:
# get individual metrics for Josephine

per_event_dfs = []
for disaster_type in ["hurricane-matthew", "santa-rosa-wildfire", "palu-tsunami"]:
    base_f1s, disaster_f1s = get_f1s_on_removed_data(disaster_type)
    base_f1s = base_f1s.rename(lambda name: name + "_base")
    disaster_f1s = disaster_f1s.rename(lambda name: name + "_disaster")
    merged_f1s = pd.concat([base_f1s, disaster_f1s])
    merged_f1s["compared_on"] = disaster_type
    per_event_dfs.append(merged_f1s)

In [47]:
pd.DataFrame(per_event_dfs).to_csv("metrics_per_disaster_event.csv", index=False)

In [48]:
per_event_dfs

[localization_f1_base                         0.77198
 damage_f1_no_damage_base                     0.36314
 damage_f1_minor_damage_base                  0.69187
 damage_f1_major_damage_base                  0.26517
 damage_f1_destroyed_base                     0.51368
 damage_f1_base                               0.40335
 score_base                                   0.51394
 localization_f1_disaster                     0.72797
 damage_f1_no_damage_disaster                 0.26639
 damage_f1_minor_damage_disaster              0.00503
 damage_f1_major_damage_disaster              0.02885
 damage_f1_destroyed_disaster                 0.41991
 damage_f1_disaster                           0.01671
 score_disaster                               0.23009
 compared_on                        hurricane-matthew
 dtype: object,
 localization_f1_base                           0.88001
 damage_f1_no_damage_base                       0.94838
 damage_f1_minor_damage_base                    0.00000
 damag

In [30]:
[name + "_base" for name in base_f1s.index]

['localization_f1_base',
 'damage_f1_no_damage_base',
 'damage_f1_minor_damage_base',
 'damage_f1_major_damage_base',
 'damage_f1_destroyed_base',
 'damage_f1_base',
 'score_base',
 'compared_on_base']

In [37]:
res = get_df_differences_on_subset_data("tsunami")

In [38]:
type(res)

pandas.core.series.Series